In [29]:
import pandas as pd

bulk_cmte_data_file = './campaign_finance_data/2019-2020_company_to_campaign_bulk_data.txt'
columns=['CMTE_ID', 'AMNDT_IND', 'RPT_TP', 'TRANSACTION_PGI', 'IMAGE_NUM', 'TRANSACTION_TP', 'ENTITY_TP', 'NAME', 'CITY', 'STATE', 'ZIP_CODE','EMPLOYER','OCCUPATION','TRANSACTION_DT','TRANSACTION_AMT','OTHER_ID','CAND_ID','TRAN_ID','FILE_NUM','MEMO_CD','MEMO_TEXT','SUB_ID']
donations_df = pd.read_csv(bulk_cmte_data_file, sep="|", header=None, low_memory=False, names=columns)

cmte_id_key_file = './campaign_finance_data/2019-2020_company_master.txt'
cmte_id_cols=['CMTE_ID', 'CMTE_NM', 'TRES_NM', 'CMTE_ST1', 'CMTE_ST2', 'CMTE_CITY', 'CMTE_ST', 'CMTE_ZIP', 'CMTE_DSGN', 'CMTE_TP', 'CMTE_PTY_AFFILIATION','CMTE_FILING_FREQ','ORG_TP','CONNECTED_ORG_NM','CAND_ID']
cmte_df = pd.read_csv(cmte_id_key_file, sep="|", header=None, low_memory=False, names=cmte_id_cols, index_col=0)

cand_id_key_file = './campaign_finance_data/2019-2020_candidate_master.txt'
cand_id_cols=['CAND_ID', 'CAND_NAME', 'CAND_PTY_AFFILIATION', 'CAND_ELECTION_YR', 'CAND_OFFICE_ST', 'CAND_OFFICE', 'CAND_OFFICE_DISTRICT', 'CAND_ICI', 'CAND_STATUS', 'CAND_PCC', 'CAND_ST1','CAND_ST2','CAND_CITY','CAND_ST','CAND_ZIP']
cand_df = pd.read_csv(cand_id_key_file, sep="|", header=None, low_memory=False, names=cand_id_cols, index_col=0)

def get_scorecard(cmte_id):
    cmte_donations_df = donations_df[donations_df.CMTE_ID == cmte_id] # Select all donations where the CMTE_ID == cmte_id_sel

    cmte_donations_df = cmte_donations_df.groupby('CAND_ID').TRANSACTION_AMT.sum()   # Sum transactions
    cmte_donations_df = pd.DataFrame(cmte_donations_df)
    cmte_donations_df = cmte_donations_df.merge(cand_df, how='left', left_index=True, right_index=True, suffixes=('',''))   # Merge candidate data into result
    
    return cmte_donations_df
    

from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
from IPython.display import display
from IPython.display import display, HTML

def f(x):
    if x == '':
        return
    cmte_id_sel = cmte_df.index[cmte_df.CMTE_NM == x]
    if cmte_id_sel.empty:
        return
    sc_df = get_scorecard(cmte_id_sel.format()[0])
    #disp_df = sc_df.reindex(columns=['CAND_NAME','TRANSACTION_AMT'])
    display(HTML(sc_df.to_html(header=False,columns=['CAND_NAME','TRANSACTION_AMT'],index=False)))

In [30]:

cb = widgets.Combobox(description='', options=[str(v) for v in cmte_df.CMTE_NM.tolist()], placeholder="Start typing here.")

interact(f, x=cb)

interactive(children=(Combobox(value='', description='x', options=('HALLMARK CARDS PAC', 'AMERICAN MEDICAL ASS…

<function __main__.f(x)>

In [31]:
from IPython.display import HTML

HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
<form action="javascript:code_toggle()"><input type="submit" value="Click here to toggle on/off the raw code."></form>''')